In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h5py
from scipy import sparse
from pathlib import Path
import scanpy as sc
import copy

import csv

In [27]:
samples = [151507, 151669, 151673]

def load_sparse_mat(filtered_filename, sample_number):
    with h5py.File(filtered_filename, 'r') as f:
        data = np.array(f['matrix']['data'])
        filtered_barcodes = np.array(f['matrix']['barcodes']).astype('U13')  # b'AAACAAGTATCTCCCA-1'  4992 columns
        _id = np.array(f['matrix']['features']['id']).astype('U13')   #  b'ENSG00000275063' 33538 rows
        name = np.array(f['matrix']['features']['name']).astype('U13')   # Gene name b'AL627309.3' 33538 rows
        indices = np.array(f['matrix']['indices'])
        indptr = np.array(f['matrix']['indptr'])
        shape = np.array(f['matrix']['shape'])
        
    barcodes = pd.DataFrame(filtered_barcodes)
    barcodes['in_filtered'] = barcodes[0].isin(filtered_barcodes).astype('int')
    m = sparse.csr_matrix((data, indices, indptr), shape=(shape[1], shape[0]))
    index = [barcodes[0].astype('str').to_list(), 
             [str(sample_number)]*len(barcodes)]
    columns = [name, _id]
    return m.toarray()

def load_data():
    col_metadata = pd.read_csv(r'../data/column_metadata.csv', index_col=0)
    row_metadata = pd.read_csv(r'../data/row_metadata.csv', index_col=0)
    filtfiles = [x for x in Path(r'..\data').glob('*filt*.h5')]
    filtfiles = [x for x in filtfiles if (str(samples[0]) in str(x)) or \
                 (str(samples[1]) in str(x)) or (str(samples[2]) in str(x))]
    dfs = []
    for filtered_file in filtfiles:
        sample_number = filtered_file.stem.split('_')[0]
        df = load_sparse_mat(filtered_filename=filtered_file, 
                             sample_number=sample_number)
        dfs.append(df)
    cdf = np.concatenate(dfs)
    adata_master = sc.AnnData(
        X=cdf,   
        obs=col_metadata.loc[col_metadata.sample_name.isin([151507, 151669, 151673])],     
        var=row_metadata)  # obs = rows      var = cols
    del cdf
    del dfs
    return adata_master
adata_master = load_data()

# Filter genes with counts less than 100
high_count_genes = adata_master.X.sum(axis=0) > 100
adata_master = adata_master[:, high_count_genes]

C:\Users\bjanderson23\anaconda3\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: X.dtype being converted to np.float32 from int32. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.


### `pd.read_csv` is slow to load the 11k x 13k normalized data csv files, but compared to `np.loadtxt` it is faster
https://stackoverflow.com/questions/8956832/python-out-of-memory-on-large-csv-file-numpy/8964779#8964779

In [41]:
normtot = copy.copy(adata_master)
sc.pp.normalize_total(normtot, exclude_highly_expressed=True, inplace=True)
normtot = normtot.T

C:\Users\bjanderson23\AppData\Roaming\Python\Python37\site-packages\scanpy\preprocessing\_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [29]:
dino = pd.read_csv(r'..\data\dino_norm.csv', index_col=0)
dino = sc.AnnData(dino.values, obs=normtot.obs, var=normtot.var)

C:\Users\bjanderson23\AppData\Roaming\Python\Python37\site-packages\scanpy\preprocessing\_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
C:\Users\bjanderson23\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  
C:\Users\bjanderson23\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  if __name__ == '__main__':


In [ ]:
sct = pd.read_csv(r'../data/SCT_norm.csv', index_col=0)
sct = sc.AnnData(sct.values, obs=normtot.obs, var=normtot.var)

In [42]:
sc.pp.log1p(normtot)

In [44]:
sc.pp.highly_variable_genes(adata=normtot, n_top_genes=1000, flavor='seurat')

In [48]:
normtot.var.loc[normtot.var['highly_variable']]

,barcode,sample_name,tissue,row,col,imagerow,imagecol,Cluster,height,width,...,SpatialDE_pool_UMAP_spatial,HVG_UMAP_spatial,pseudobulk_UMAP_spatial,markers_UMAP_spatial,spatialLIBD,ManualAnnotation,highly_variable,means,dispersions,dispersions_norm
AAACGGTTGCGAACTG-1,AAACGGTTGCGAACTG-1,151507,1,67,59,474.707487,279.567966,2,600,600,...,3,5,1,6,WM,NaN,True,0.200118,2.505050,1.916875
AAACTGCTGGCTCCAA-1,AAACTGCTGGCTCCAA-1,151507,1,45,67,356.165629,304.725483,1,600,600,...,3,1,1,2,L4,NaN,True,0.209708,2.888867,1.570408
AAAGTTGACTCCCGTA-1,AAAGTTGACTCCCGTA-1,151507,1,42,96,340.279040,394.599474,5,600,600,...,6,5,2,1,L3,NaN,True,0.210530,2.888629,1.567817
AAATCGTGTACCACAA-1,AAATCGTGTACCACAA-1,151507,1,44,56,350.675080,270.657075,1,600,600,...,1,1,1,1,L5,NaN,True,0.210373,2.863396,1.293562
AAATGGCATGTCTTGT-1,AAATGGCATGTCTTGT-1,151507,1,13,69,183.708377,311.431154,2,600,600,...,1,4,2,1,L1,NaN,True,0.217444,3.270350,1.327495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTCCTTTCTGTGTTGC-1.8,TTCCTTTCTGTGTTGC-1,151673,1,11,25,169.081914,203.555363,1,600,600,...,1,2,4,1,L1,NaN,True,0.223433,3.475324,1.901336
TTCGCACTGTACGACA-1.8,TTCGCACTGTACGACA-1,151673,1,61,59,439.423958,306.975708,1,600,600,...,1,1,3,3,L6,NaN,True,0.201750,2.514798,1.992843
TTCGGACTGGGCATGG-1.3,TTCGGACTGGGCATGG-1,151673,1,68,6,476.012618,142.439249,8,600,600,...,7,5,6,3,WM,NaN,True,0.195113,2.346234,1.533431
TTCTCTTACAGGTGAT-1.5,TTCTCTTACAGGTGAT-1,151673,1,62,8,443.699385,148.874893,8,600,600,...,7,5,7,3,WM,NaN,True,0.195181,2.410610,1.890723


In [33]:
normtot.X.max()

330.25